# Quick Start

## 步骤 0：准备工作

首先，在环境中安装 FlagEmbedding。

`pip install -U FlagEmbedding`

以下是一个非常小的语料库，仅包含 10 个句子，我们将用它作为数据集。

每个句子都是对某一领域著名人物的简明描述。

In [28]:
corpus = [
    "迈克尔·杰克逊是一位传奇的流行音乐偶像，以其破纪录的音乐和舞蹈创新而闻名。",
    "李飞飞是斯坦福大学教授，通过ImageNet项目彻底改变了计算机视觉领域。",
    "布拉德·皮特是一位多才多艺的演员兼制片人，因在《搏击俱乐部》和《好莱坞往事》等影片中的角色而闻名。",
    "杰弗里·辛顿是人工智能领域的奠基性人物，因其在深度学习方面的贡献而荣获图灵奖。",
    "埃米纳姆是一位著名的说唱歌手，是有史以来销量最高的音乐艺术家之一。",
    "泰勒·斯威夫特是一位格莱美获奖的创作歌手，以其叙事性强的音乐而著称。",
    "萨姆·奥尔特曼担任OpenAI的首席执行官，主导了GPT系列的惊人成果，并致力于推动安全且有益的人工智能发展。",
    "摩根·弗里曼是一位备受赞誉的演员，以其独特嗓音和多样化的角色而闻名。",
    "吴恩达通过在Coursera和斯坦福大学开设公开课，将人工智能知识传播至全球。",
    "小罗伯特·唐尼是一位标志性演员，最广为人知的角色是在漫威电影宇宙中饰演钢铁侠。",
]

我们想知道这些人中谁可能是神经网络领域的专家，以及他/她是谁。

因此，我们生成了以下查询：

In [29]:
query = "谁可能是神经网络领域的专家？"

## 步骤 1：文本 → 嵌入向量

首先，我们使用 [BGE 嵌入模型](https://huggingface.co/BAAI/bge-base-zh-v1.5) 为语料库中的句子生成嵌入向量。

注：国内用户手动下载。

```powershell
# 激活虚拟环境
.\.venv\Scripts\Activate.ps1

# 设置镜像
$env:HF_ENDPOINT = "https://hf-mirror.com"

# 使用 huggingface-cli 下载（确保已安装 huggingface_hub >= 0.14）
huggingface-cli download --resume-download BAAI/bge-base-zh-v1.5 --local-dir ./.models/bge-base-zh-v1.5 --local-dir-use-symlinks False
```

In [30]:
import os
from FlagEmbedding import FlagModel
try:
    # 脚本环境
    current_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # Notebook / 交互式环境
    current_dir = os.getcwd()  # 使用当前工作目录

# 推断项目根目录：上两级（zh -> Tutorials -> FlagEmbedding-cn）
project_root = os.path.dirname(os.path.dirname(current_dir))

# 拼接模型路径
model_path = os.path.join(project_root, ".models", "bge-base-zh-v1.5")

print("Loading model from:", model_path)
# 获取 BGE 嵌入模型
model = FlagModel(model_path,
                  query_instruction_for_retrieval="为检索相关段落而对这个句子进行表示：",
                  use_fp16=True)

# 获取查询和语料库的嵌入
corpus_embeddings = model.encode(corpus)
query_embedding = model.encode(query)

Loading model from: e:\git\FlagEmbedding-cn\.models\bge-base-zh-v1.5


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


每个句子的嵌入是一个长度为 768 的向量。

In [31]:
print("查询嵌入的形状：", query_embedding.shape)
print("语料库嵌入的形状：", corpus_embeddings.shape)

查询嵌入的形状： (768,)
语料库嵌入的形状： (10, 768)


运行以下打印语句，查看查询嵌入向量的前 10 个元素。

In [32]:
print(query_embedding[:10])

[-0.02136574 -0.02101879  0.00016666  0.01245924  0.04682833  0.03393672
  0.0190743  -0.04130525 -0.00147657  0.04505555]


## 步骤 2：计算相似度

现在，我们已经获得了查询和语料库的嵌入向量。下一步是计算查询与语料库中每个句子之间的相似度。这里我们使用点积（内积）作为相似度度量指标。

In [33]:
sim_scores = query_embedding @ corpus_embeddings.T
print(sim_scores)

[0.27556407 0.51574427 0.3341428  0.54028046 0.3247512  0.33678967
 0.4857601  0.32122058 0.43284306 0.34145787]


结果是一个得分列表，表示查询与以下各项的相似度：[句子0, 句子1, 句子2, ...]

## 步骤 3：排序

在获得查询与语料库中每个句子的相似度得分后，我们可以将这些得分从大到小进行排序。

In [34]:
# 获取按排序顺序排列的索引
sorted_indices = sorted(range(len(sim_scores)), key=lambda k: sim_scores[k], reverse=True)
print(sorted_indices)

[3, 1, 6, 8, 9, 5, 2, 4, 7, 0]


根据排序结果，索引为 3 的句子是对我们查询 “谁可能是神经网络领域的专家？” 的最佳回答。

而这位专家正是 杰弗里·辛顿！

In [35]:
print(corpus[3])

杰弗里·辛顿是人工智能领域的奠基性人物，因其在深度学习方面的贡献而荣获图灵奖。


根据索引的排序顺序，我们可以打印出这个小型检索系统得到的人物排名。

In [36]:
# 依次按降序打印得分及对应的句子

for i in sorted_indices:
    print(f"得分 {sim_scores[i]:.3f}: \"{corpus[i]}\"")

得分 0.540: "杰弗里·辛顿是人工智能领域的奠基性人物，因其在深度学习方面的贡献而荣获图灵奖。"
得分 0.516: "李飞飞是斯坦福大学教授，通过ImageNet项目彻底改变了计算机视觉领域。"
得分 0.486: "萨姆·奥尔特曼担任OpenAI的首席执行官，主导了GPT系列的惊人成果，并致力于推动安全且有益的人工智能发展。"
得分 0.433: "吴恩达通过在Coursera和斯坦福大学开设公开课，将人工智能知识传播至全球。"
得分 0.341: "小罗伯特·唐尼是一位标志性演员，最广为人知的角色是在漫威电影宇宙中饰演钢铁侠。"
得分 0.337: "泰勒·斯威夫特是一位格莱美获奖的创作歌手，以其叙事性强的音乐而著称。"
得分 0.334: "布拉德·皮特是一位多才多艺的演员兼制片人，因在《搏击俱乐部》和《好莱坞往事》等影片中的角色而闻名。"
得分 0.325: "埃米纳姆是一位著名的说唱歌手，是有史以来销量最高的音乐艺术家之一。"
得分 0.321: "摩根·弗里曼是一位备受赞誉的演员，以其独特嗓音和多样化的角色而闻名。"
得分 0.276: "迈克尔·杰克逊是一位传奇的流行音乐偶像，以其破纪录的音乐和舞蹈创新而闻名。"


从排名结果来看，不出所料，查询与杰弗里·辛顿（Geoffrey Hinton）和李飞飞（Fei-Fei Li）描述之间的相似度得分远高于其他人，其次是吴恩达（Andrew Ng）和萨姆·奥尔特曼（Sam Altman）。

尽管查询中的关键词短语 “神经网络” 并未出现在任何描述中，但 BGE 嵌入模型依然足够强大，能够准确捕捉查询与语料库文本之间的语义含义。

## 步骤 4：评估

我们已经看到嵌入模型在“神经网络”查询上的表现相当不错。那么它在更一般情况下的整体质量如何呢？

让我们构建一个非常小的查询数据集，并为每个查询提供对应的标准答案（ground truth）。注意，这些标准答案是语料库中句子的索引。

In [37]:
queries = [
    "谁可能是神经网络领域的专家？",
    "谁可能获得过格莱美奖？",
    "获得过奥斯卡奖",
    "最著名的女歌手之一。",
    "AlexNet 的发明者",
]

In [38]:
ground_truth = [
    [1, 3],
    [0, 4, 5],
    [2, 7, 9],
    [5],
    [3],
]

在这里，我们重复上述步骤，为每个查询获取预测的排序结果。

In [39]:
# 使用 BGE 模型为所有查询生成嵌入向量。
queries_embedding = model.encode(queries)
# 计算相似度得分
scores = queries_embedding @ corpus_embeddings.T
# 获取最终的排名
rankings = [sorted(range(len(sim_scores)), key=lambda k: sim_scores[k], reverse=True) for sim_scores in scores]
rankings

[[3, 1, 6, 8, 9, 5, 2, 4, 7, 0],
 [5, 3, 2, 7, 9, 4, 0, 1, 6, 8],
 [5, 2, 3, 7, 9, 1, 4, 0, 8, 6],
 [4, 5, 0, 7, 3, 2, 9, 8, 6, 1],
 [1, 3, 6, 8, 0, 4, 5, 9, 2, 7]]

平均倒数排名（[MRR](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)）是信息检索中广泛使用的一种评估指标。在这里，我们使用它来粗略了解我们系统的性能。

In [40]:
def MRR(preds, labels, cutoffs):
    mrr = [0 for _ in range(len(cutoffs))]
    for pred, label in zip(preds, labels):
        for i, c in enumerate(cutoffs):
            for j, index in enumerate(pred):
                if j < c and index in label:
                    mrr[i] += 1/(j+1)
                    break
    mrr = [k/len(preds) for k in mrr]
    return mrr

我们选择将 1 和 5 作为截断点（cutoffs），分别得到了 0.4 和 0.7 的结果。这表明，在这个小型数据集上，我们的嵌入模型表现良好。

In [41]:
cutoffs = [1, 5]
mrrs = MRR(rankings, ground_truth, cutoffs)
for i, c in enumerate(cutoffs):
    print(f"MRR@{c}: {mrrs[i]}")

MRR@1: 0.4
MRR@5: 0.7
